<a href="https://colab.research.google.com/github/dsandeep0138/HackerEarth-Challenges/blob/master/flower_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q kaggle

In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"dsandeep97","key":"91ed93d9f57d24d723707d120074f706"}'}

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
 !chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d rednivrug/flower-recognition-he

 99% 897M/905M [00:18<00:00, 70.0MB/s]
100% 905M/905M [00:18<00:00, 50.8MB/s]


In [0]:
!unzip flower-recognition-he.zip

In [0]:
import gc, cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau

from keras import applications
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, BatchNormalization, MaxPooling2D, Dropout, Flatten, AveragePooling2D
from keras.preprocessing.image import ImageDataGenerator

In [0]:
traindf = pd.read_csv('data/train.csv', dtype=str)
testdf = pd.read_csv('data/test.csv', dtype=str)

In [0]:
traindf['image_id'] = traindf['image_id'] + ".jpg"
testdf['image_id'] = testdf['image_id'] + ".jpg"

In [19]:
datagen = ImageDataGenerator(rescale=1./255.,
                             rotation_range=20,
                             width_shift_range=0.2, 
                             height_shift_range=0.2,
                             horizontal_flip=True)

train_generator = datagen.flow_from_dataframe(
dataframe=traindf,
directory="./data/train/",
x_col="image_id",
y_col="category",
subset="training",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224, 224))

Found 18540 validated image filenames belonging to 102 classes.


In [20]:
test_datagen=ImageDataGenerator(rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
dataframe=testdf,
directory="./data/test/",
x_col="image_id",
y_col=None,
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(224, 224))

Found 2009 validated image filenames.


In [21]:
densenet_model = applications.densenet.DenseNet121(weights = "imagenet", include_top=False, input_shape = (224, 224, 3))

for layer in densenet_model.layers[:-4]:
    layer.trainable=False
    
input = Input(shape=(224, 224, 3), name = 'image_input')
output_densenet_conv = densenet_model(input)

x = AveragePooling2D(pool_size=(3,3))(output_densenet_conv)

x = Flatten()(x)
x = Dense(4096, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(1024, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)

x = Dense(102, activation='softmax')(x)

model = Model(input=input, output=x)

model.summary()


29089792/29084464 [==============================] - 1s 0us/step
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
average_pooling2d_1 (Average (None, 2, 2, 1024)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 4096)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 4096)              16781312  
_________________________________________________________________
batch_normalization_12 (Batc (None, 4096)              16384     
___________________________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("de...)`


In [0]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08, decay=0.0),
              metrics=['accuracy'])

In [23]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    epochs=50,
                    verbose=1)

Epoch 1/50
579/579 [==============================] - 238s 411ms/step - loss: 1.4663 - acc: 0.6533
Epoch 2/50
579/579 [==============================] - 230s 397ms/step - loss: 0.5166 - acc: 0.8660
Epoch 3/50
579/579 [==============================] - 228s 393ms/step - loss: 0.3488 - acc: 0.9067
Epoch 4/50
579/579 [==============================] - 227s 392ms/step - loss: 0.2703 - acc: 0.9250
Epoch 5/50
579/579 [==============================] - 224s 388ms/step - loss: 0.2387 - acc: 0.9346
Epoch 6/50
579/579 [==============================] - 223s 384ms/step - loss: 0.2003 - acc: 0.9431
Epoch 7/50
579/579 [==============================] - 224s 386ms/step - loss: 0.1720 - acc: 0.9515
Epoch 8/50
579/579 [==============================] - 223s 385ms/step - loss: 0.1599 - acc: 0.9540
Epoch 9/50
579/579 [==============================] - 223s 385ms/step - loss: 0.1420 - acc: 0.9594
Epoch 10/50
579/579 [==============================] - 224s 386ms/step - loss: 0.1357 - acc: 0.9614
Epoch 11/

In [24]:
test_generator.reset()
pred = model.predict_generator(test_generator,
                               steps=STEP_SIZE_TEST,
                               verbose=1)

2009/2009 [==============================] - 34s 17ms/step


In [0]:
predicted_class_indices=np.argmax(pred, axis=1)

In [0]:
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [0]:
filenames=test_generator.filenames
filenames=[f.split('.')[0] for f in filenames]

results=pd.DataFrame({"image_id":filenames,
                      "category":predictions})
results = results.sort_values(by = ['image_id'], ascending = [True])
results.to_csv("results.csv", index=False)